Solar Radiation Measurement Data EDA

In [16]:
# Load the data 
import pandas as pd

togo_data = pd.read_csv('../../data/data/togo-dapaong_qc.csv')

benin_data = pd.read_csv('../../data/data/benin-malanville.csv')

sierra_leone_data = pd.read_csv('../../data/data/sierraleone-bumbuna.csv')

In [17]:
# initial data exploration

print(togo_data.head())
print(benin_data.head())
print(sierra_leone_data.head())

print(togo_data.info())
print(benin_data.info())
print(sierra_leone_data.info())


          Timestamp  GHI  DNI  DHI  ModA  ModB  Tamb    RH   WS  WSgust  \
0  2021-10-25 00:01 -1.3  0.0  0.0   0.0   0.0  24.8  94.5  0.9     1.1   
1  2021-10-25 00:02 -1.3  0.0  0.0   0.0   0.0  24.8  94.4  1.1     1.6   
2  2021-10-25 00:03 -1.3  0.0  0.0   0.0   0.0  24.8  94.4  1.2     1.4   
3  2021-10-25 00:04 -1.2  0.0  0.0   0.0   0.0  24.8  94.3  1.2     1.6   
4  2021-10-25 00:05 -1.2  0.0  0.0   0.0   0.0  24.8  94.0  1.3     1.6   

   WSstdev     WD  WDstdev   BP  Cleaning  Precipitation  TModA  TModB  \
0      0.4  227.6      1.1  977         0            0.0   24.7   24.4   
1      0.4  229.3      0.7  977         0            0.0   24.7   24.4   
2      0.3  228.5      2.9  977         0            0.0   24.7   24.4   
3      0.3  229.1      4.6  977         0            0.0   24.7   24.4   
4      0.4  227.5      1.6  977         0            0.0   24.7   24.4   

   Comments  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  
          Timestamp 

In [18]:
# Missing values check
print(togo_data.isnull().sum())
print(benin_data.isnull().sum())
print(sierra_leone_data.isnull().sum())

Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525600
dtype: int64
Timestamp             0
GHI                   0
DNI                   0
DHI                   0
ModA                  0
ModB                  0
Tamb                  0
RH                    0
WS                    0
WSgust                0
WSstdev               0
WD                    0
WDstdev               0
BP                    0
Cleaning              0
Precipitation         0
TModA                 0
TModB                 0
Comments         525600
dtype: int64
Timestamp             0
GHI                   0
DNI           

In [20]:
import numpy as np

def clean_negative_values(df):
    for col in ['GHI', 'DNI', 'DHI']:
        df[col] = df[col].apply(lambda x: np.nan if x < 0 else x)
        df[col] = df[col].apply(lambda x: np.nan if x == 0 else x)
    return df

togo_cleaned = clean_negative_values(togo_data)
benin_cleaned = clean_negative_values(benin_data)
sierra_leone_cleaned = clean_negative_values(sierra_leone_data)


In [21]:
#Dropping the Empty Comments Column
togo_cleaned = togo_cleaned.drop(columns=['Comments'])
benin_cleaned = benin_cleaned.drop(columns=['Comments'])
sierra_leone_cleaned = sierra_leone_cleaned.drop(columns=['Comments'])